Authors: Alexis Raja Brachet, Louis Chirol, Sophia Chirrane, Tanguy Olympie

# Graph classification on the COLLAB dataset

https://paperswithcode.com/sota/graph-classification-on-collab


Install required packages and import libraries

In [ ]:
!pip install dgl
!pip install transformers

import seaborn as sns
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

Import the dataset

In [ ]:
# https://docs.dgl.ai/en/0.8.x/api/python/dgl.DGLGraph.html

import dgl.data
dataset = dgl.data.TUDataset('COLLAB')#, directory='~/datasets/TUDataset')

print(f'Dataset: {dataset}:')
print('max_num_node : ', dataset.max_num_node)
print('num_labels : ',dataset.num_labels)

# Gather some statistics about the first graph.
data = dataset[0]  # Get the first graph object.
print(f'Number of nodes: {data[0].num_nodes()}')
print(f'Number of edges: {data[0].num_edges()}')
print(f'Average node degree: {data[0].num_edges() / data[0].num_nodes():.2f}')


# Getting to know the data

In [ ]:
l = len(dataset) #extract dataset length = 5000

graph, label = dataset[0]
graph1, label = dataset[1]
print(l)

5000


In [ ]:
nx_g = dgl.to_networkx(graph)
nx_g1 = dgl.to_networkx(graph1)

### Some edges are represented multiple times
l=[]
for (a,b,w) in nx_g.edges(data=True):
  if a==23:
    l.append((a,b,w))
l[:10]

nx.draw_networkx(nx_g)

#### EDA

Let's explore some basic properties of the graphs in the dataset

In [ ]:
labels = [str(g[1].item()) for g in dataset]
num_nodes = [g[0].num_nodes() for g in dataset]
num_edges = [g[0].num_edges() for g in dataset]
degrees = [g[0].in_degrees().float().mean().item() for g in dataset]

diameters = []

for g in tqdm(dataset):
    graph = g[0]
    # Compute the shortest path between all pairs of nodes
    dist = nx.floyd_warshall_numpy(graph.to_networkx())
    # Compute the diameter of the graph
    diameters.append(dist.max())


A macro visualization of the data

In [ ]:
# Explore the dataset: statistics over the number of nodes and edges, and the number of classes.
print('Number of graphs:', len(dataset))
print('Number of classes:', dataset.num_classes)
print('Average number of nodes:', np.mean(num_nodes))
print('Average number of edges:', np.mean(num_edges))
print('Average node degree:', np.mean(degrees))
print('Average diameter:', np.mean(diameters))

# Same but in log scale with seaborn
sns.set_style('whitegrid')

fig, ax = plt.subplots(2, 3, figsize=(15, 6))
sns.histplot(num_nodes, bins=20, ax=ax[0,0])
ax[0,0].set_xlabel('Number of nodes')
ax[0,0].set_ylabel('Number of graphs')
ax[0,0].set_yscale('log')
# Plot average as a vertical line
ax[0,0].axvline(np.mean(num_nodes), color='red', linestyle='--')

sns.histplot(num_edges, bins=20, ax=ax[0,1])
ax[0,1].set_xlabel('Number of edges')
ax[0,1].set_ylabel('Number of graphs')
ax[0,1].set_yscale('log')
# Plot average as a vertical line
ax[0,1].axvline(np.mean(num_edges), color='red', linestyle='--')


sns.histplot(degrees, bins=20, ax=ax[0,2])
ax[0,2].set_xlabel('Average node degree')
ax[0,2].set_ylabel('Number of graphs')
ax[0,2].axvline(np.mean(degrees), color='red', linestyle='--')

sns.histplot(diameters, bins=20, ax=ax[1,0])
ax[1,0].set_xlabel('Diameter')
ax[1,0].set_ylabel('Number of graphs')


sns.histplot(labels, bins=dataset.num_classes, ax=ax[1,1])
ax[1,1].set_xlabel('Class')
ax[1,1].set_ylabel('Number of graphs')

plt.show()

A per class visualization

In [ ]:
df = pd.DataFrame({'labels': labels, 'num_nodes': num_nodes, 'num_edges': num_edges,
                   'degrees': degrees})
df['diameter'] = diameters
df.diameter = df.diameter.astype(str)

fig, ax = plt.subplots(1, 4, figsize=(20, 4))
sns.boxplot(x='labels', y='num_nodes', data=df, ax=ax[0],
            palette='Set2')
ax[0].set_xlabel('Class')
ax[0].set_ylabel('Number of nodes')
ax[0].set_yscale('log')
sns.boxplot(x='labels', y='num_edges', data=df, ax=ax[1],
            palette='Set2')
ax[1].set_xlabel('Class')
ax[1].set_ylabel('Number of edges')
ax[1].set_yscale('log')
sns.boxplot(x='labels', y='degrees', data=df, ax=ax[2],
            palette='Set2')
ax[2].set_xlabel('Class')
ax[2].set_ylabel('Average node degree')
ax[2].set_yscale('log')
sns.histplot(data=df, x='diameter', hue='labels', multiple='stack',
             bins=20, palette='Set2', discrete=True, shrink=0.8, ax=ax[3])
ax[3].set_xlabel('Diameter')
ax[3].set_ylabel('Count')

plt.show()


# Preprocessing

Conversion to a weighted simple graph taken from

https://stackoverflow.com/questions/15590812/networkx-convert-multigraph-into-simple-graph-with-weighted-edges

In [ ]:
# create weighted graph from multigraph
def multi_graph_to_weighted(multigraph):
  G = nx.Graph()
  for u,v,data in multigraph.edges(data=True):
      w = data['weight'] if 'weight' in data else 1.0
      if G.has_edge(u,v):
          G[u][v]['weight'] += w
      else:
          G.add_edge(u, v, weight=w)
  return G

# Machine Learning techniques

## Feature extraction

In [ ]:
labels=[]
for i in tqdm(range(len(dataset))): #len(dataset)
  graph,label=dataset[i]
  labels.append(int(label))

100%|██████████| 5000/5000 [00:00<00:00, 121429.72it/s]


In [ ]:

clustering_coefficients=[]
avg_nmbr_edges=[]
densities=[]
mean_average_degree_connectivities=[]
std_average_degree_connectivities=[]
diameters=[]
max_number_connected_components=[]
average_shortest_path_lengths=[]
number_of_edges=[]
number_of_nodes=[]


for i in tqdm(range(len(dataset))): #len(dataset)
  graph,label=dataset[i]
  nx_graph=dgl.to_networkx(graph)
  weighted_graph=multi_graph_to_weighted(nx_graph)

  clustering_coefficients.append(nx.average_clustering(weighted_graph))

  avg_nmbr_edges.append(nx.number_of_edges(weighted_graph)/nx.number_of_nodes(weighted_graph))  

  densities.append(nx.density(weighted_graph))

  average_degree_connectivities=nx.average_degree_connectivity(weighted_graph)

  mean_average_degree_connectivities.append(np.mean(np.array(list(average_degree_connectivities.values()))/weighted_graph.number_of_nodes()))
  std_average_degree_connectivities.append(np.std(np.array(list(average_degree_connectivities.values()))/weighted_graph.number_of_nodes()))
  diameters.append(nx.diameter(weighted_graph))

  max_number_connected_components.append(len(max(nx.connected_components(weighted_graph), key=len)))

  average_shortest_path_lengths.append(nx.average_shortest_path_length(weighted_graph))

  number_of_edges.append(nx.number_of_edges(weighted_graph))

  number_of_nodes.append(nx.number_of_nodes(weighted_graph))

df=pd.DataFrame()
df['clustering_coefficients']=clustering_coefficients
df['avg_nmbr_edges']=avg_nmbr_edges
df['densities']=densities
df['mean_average_degree_connectivities']=mean_average_degree_connectivities ###To be tweaked to be usable
df['std_average_degree_connectivities']=std_average_degree_connectivities ###To be tweaked to be usable
df['diameters']=diameters
df['max_number_connected_components']=max_number_connected_components
df['average_shortest_path_lengths']=average_shortest_path_lengths
df['number_of_edges']=number_of_edges
df['number_of_nodes']=number_of_nodes

df['labels']=labels
df.tail(10)

100%|██████████| 5000/5000 [06:41<00:00, 12.45it/s]


In [ ]:
df.columns

Index(['clustering_coefficients', 'avg_nmbr_edges', 'densities', 'diameters',
       'max_number_connected_components', 'average_shortest_path_lengths',
       'number_of_edges', 'number_of_nodes', 'labels',
       'mean_average_degree_connectivities',
       'std_average_degree_connectivities'],
      dtype='object')

In [ ]:
features=['clustering_coefficients', 'avg_nmbr_edges', 'densities', 'diameters',
       'max_number_connected_components', 'average_shortest_path_lengths',
       'number_of_edges', 'number_of_nodes','mean_average_degree_connectivities',
       'std_average_degree_connectivities']

from sklearn.model_selection import train_test_split

X=df[features]
y=df['labels']

X_train_validation, X_test, y_train_validation, y_test = train_test_split(X,y,test_size=0.2,random_state=1)

X_train, X_validation, ytrain, y_validation = train_test_split(X_train_validation,y_train_validation,test_size=0.1,random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression

LogisticClassifier=LogisticRegression()
LogisticClassifier.fit(X_train_validation, y_train_validation)

y_pred3 = LogisticClassifier.predict(X_test)
print("Logistic Regression Classifier : ")
print('Accuracy:', accuracy_score(y_test, y_pred3))
print('Classification report:', classification_report(y_test, y_pred3))

In [ ]:
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score

SVCclassifier = svm.SVC(kernel='linear', probability=True)
SVCclassifier.fit(X_train_validation, y_train_validation)


y_pred = SVCclassifier.predict(X_test)

print("Support Vector Machine Classifier : ")
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification report:', classification_report(y_test, y_pred))


KeyboardInterrupt: ignored

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RandForClassifier=RandomForestClassifier()
RandForClassifier.fit(X_train_validation, y_train_validation)

y_pred2 = RandForClassifier.predict(X_test)

print("Logistic Regression Classifier : ")
print('Accuracy:', accuracy_score(y_test, y_pred2))
print('Classification report:', classification_report(y_test, y_pred2))

Accuracy: 0.764
Classification report:               precision    recall  f1-score   support

           0       0.76      0.84      0.80       524
           1       0.60      0.67      0.63       144
           2       0.87      0.69      0.77       332

    accuracy                           0.76      1000
   macro avg       0.74      0.73      0.73      1000
weighted avg       0.77      0.76      0.76      1000



In [ ]:
print('Importance scores of each feature : ')

for i in range(len(features)):
  print(features[i] + ' : ' + str(RandForClassifier.feature_importances_[i]))


Importances of each feature : 
clustering_coefficients : 0.12350499486313828
avg_nmbr_edges : 0.2037594017335099
densities : 0.09763902829861067
diameters : 0.002269931224985741
max_number_connected_components : 0.05298391247248613
average_shortest_path_lengths : 0.10188465635271458
number_of_edges : 0.15640391636691753
number_of_nodes : 0.05661566762329325
mean_average_degree_connectivities : 0.10742706240834436
std_average_degree_connectivities : 0.09751142865599963


## Distance based methods

In [ ]:
graphs_0 , _ = dataset[:100]
graphs_1 , _ = dataset[3000:3100] 
graphs_2 , _ = dataset[4800:4900] 

In [ ]:
import networkx as nx
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from networkx.algorithms.similarity import graph_edit_distance
from sklearn.metrics import accuracy_score
from sklearn.manifold import SpectralEmbedding
from sklearn.metrics.pairwise import cosine_similarity

# create example graphs and labels

X = graphs_0 + graphs_1 + graphs_2
X = [dgl.to_networkx(graph) for graph in X]
X = [ multi_graph_to_weighted(nx_graph) for nx_graph in X]
y = np.concatenate((np.zeros(len(graphs_0)), np.ones(len(graphs_1)), 2*np.ones(len(graphs_2))))

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# define a function to comut distance between two graphs
def ged_distance(G1, G2):
    return graph_edit_distance(G1, G2)

def rwd_distance(G1,G2): #OK
    rw1 = SpectralEmbedding(n_components=1, affinity='precomputed').fit_transform(nx.to_numpy_array(G1))
    rw2 = SpectralEmbedding(n_components=1, affinity='precomputed').fit_transform(nx.to_numpy_array(G2))
    rwd = cosine_similarity(rw1, rw2)[0, 0]
    return rwd

def short_path_distance(G1,G2): #OK
    return nx.average_shortest_path_length(G1) - nx.average_shortest_path_length(G2)

def graphlet_distance(G1,G2):
    X = [graph_from_networkx(G1), graph_from_networkx(G2)]
    # Compute graphlet kernel
    kernel = GraphletSampling()
    K = kernel.fit_transform(X)
    return K

def wl_distance(G1,G2):
    X = [graph_from_networkx(G1), graph_from_networkx(G2)]
    # Compute graphlet kernel
    kernel = ShortestPath(normalize=True)
    K = kernel.fit_transform(np.array(X))
    return K

def diameter_distance(G1,G2): #OK
    return abs(nx.diameter(G1) - nx.diameter(G2))

class KNN:
    """K-Nearest Neighbors classifier"""
    
    def __init__(self, k=5, metric =ged_distance ):
        self.k = k
        self.metric = metric

        
    def fit(self, X, y):
        self.X_ = X
        self.y_ = y
        
    def predict(self, X):
        y_pred = []
        for k in tqdm(range(len(X))):
            x = X[k]
            # compute distances to all training samples
            distances = [self._distance(x, x_train) for x_train in self.X_]
            # find k nearest neighbors and their corresponding labels
            k_nearest = np.argsort(distances)[:self.k]
            k_nearest_labels = self.y_[k_nearest]
            
            # predict the label of the sample based on the majority vote of the neighbors
            classes, counts = np.unique(k_nearest_labels, return_counts=True)
            max_count = np.max(counts)
            candidates = classes[counts == max_count]
            y_pred.append(np.random.choice(candidates))
        
        return y_pred
    
    def _distance(self, x1, x2):
        return self.metric(x1,x2)


In [ ]:
# train KNN classifier and make predictions on test set
clf = KNN(k=3, metric = diameter_distance)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(y_pred, y_test)
# compute accuracy score
acc = accuracy_score(y_test, y_pred)

print("Accuracy:", acc)

  2%|▏         | 1/60 [00:22<22:30, 22.89s/it]


KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import balanced_accuracy_score
b_acc = balanced_accuracy_score(y_test, y_pred)
print(b_acc)

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Assuming y_true and y_pred are your true and predicted labels, respectively
cm = confusion_matrix(y_test, y_pred)

# Define the class names (if any)
class_names = ["Class 1", "Class 2", "Class 3"]

# Normalize the confusion matrix if desired
normalize = True
if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Create the figure and subplot
fig, ax = plt.subplots(figsize=(8, 8))

# Plot the confusion matrix
im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)

# Add the class names to the axis labels
ax.set(xticks=np.arange(cm.shape[1]),
       yticks=np.arange(cm.shape[0]),
       xticklabels=class_names, yticklabels=class_names,
       title="Confusion matrix",
       ylabel="True label",
       xlabel="Predicted label")

# Rotate the tick labels and set their alignment
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over the data and create the annotations
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, "{:.2f}".format(cm[i, j]),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black")

# Display the plot
plt.show()


NameError: ignored

## GNN

In this section, the following architectures have been explored :

- GraphSAGE
- GAT

We first derive the Node2Vec algorithm, then define the datalaoders to get the training, validation and test sets. Finally, both architectures are implemented and trained.


Node2Vec algorithm

In [ ]:
from gensim.models import Word2Vec
#from node2vec import Node2Vec
from scipy.sparse import *
from scipy.stats.stats import pearsonr
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, auc
from sklearn.preprocessing import LabelEncoder 

# Functions seen in previous lab 

def generate_samples(graph, train_set_ratio):
    """
    Graph pre-processing step required to perform supervised link prediction
    Create training and test sets
    """
        
    # --- Step 0: The graph must be connected ---
    if nx.is_connected(graph) is not True:
        raise ValueError("The graph contains more than one connected component!")
       
    
    # --- Step 1: Generate positive edge samples for testing set ---
    residual_g = graph.copy()
    test_pos_samples = []
      
    # Store the shuffled list of current edges of the graph
    edges = list(residual_g.edges())
    np.random.shuffle(edges)
    
    # Define number of positive test samples desired
    test_set_size = int((1.0 - train_set_ratio) * graph.number_of_edges())
    train_set_size = graph.number_of_edges() - test_set_size
    num_of_pos_test_samples = 0
    
    # Remove random edges from the graph, leaving it connected
    # Fill in the blanks
    for edge in edges:
        
        # Remove the edge
        residual_g.remove_edge(edge[0], edge[1])
        
        # Add the removed edge to the positive sample list if the network is still connected
        if nx.is_connected(residual_g):
            num_of_pos_test_samples += 1
            test_pos_samples.append(edge)
        # Otherwise, re-add the edge to the network
        else: 
            residual_g.add_edge(edge[0], edge[1])
        
        # If we have collected enough number of edges for testing set, we can terminate the loop
        if num_of_pos_test_samples == test_set_size:
            break
    
    # Check if we have the desired number of positive samples for testing set 
    if num_of_pos_test_samples != test_set_size:
        raise ValueError("Enough positive edge samples could not be found!")

        
    # --- Step 2: Generate positive edge samples for training set ---
    # The remaining edges are simply considered for positive samples of the training set
    train_pos_samples = list(residual_g.edges())
        
        
    # --- Step 3: Generate the negative samples for testing and training sets ---
    # Fill in the blanks
    non_edges = list(nx.non_edges(graph))
    np.random.shuffle(non_edges)
    
    train_neg_samples = non_edges[:train_set_size] 
    test_neg_samples = non_edges[train_set_size:train_set_size + test_set_size]

    
    # --- Step 4: Combine sample lists and create corresponding labels ---
    # For training set
    train_samples = train_pos_samples + train_neg_samples
    train_labels = [1 for _ in train_pos_samples] + [0 for _ in train_neg_samples]
    # For testing set
    test_samples = test_pos_samples + test_neg_samples
    test_labels = [1 for _ in test_pos_samples] + [0 for _ in test_neg_samples]
    
    return residual_g, train_samples, train_labels, test_samples, test_labels


def edge_prediction(node2embedding, train_samples, test_samples, train_labels, test_labels, feature_func=None, plot_roc=True):
    
    # --- Construct feature vectors for edges ---
    if feature_func is None:
        feature_func = lambda x,y: abs(x-y)
    
    # Fill in the blanks
    train_features = [feature_func(node2embedding[edge[0]], node2embedding[edge[1]]) for edge in train_samples]
    test_features = [feature_func(node2embedding[edge[0]], node2embedding[edge[1]]) for edge in test_samples]
    
    # --- Build the model and train it ---
    # Fill in the blanks
    clf = LogisticRegression()
    clf.fit(train_features, train_labels)

    train_preds = clf.predict_proba(train_features)[:, 1]
    test_preds = clf.predict_proba(test_features)[:, 1]

    # --- Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from predictions ---
    # Fill in the blanks
    fpr, tpr, _ = roc_curve(test_labels, test_preds)
    roc_auc = auc(fpr, tpr)

    if not plot_roc:
        return roc_auc
    
    plt.figure(figsize=(6,6))
    plt.plot(fpr, tpr, color='darkred', label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='lightgray', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend(loc="lower right")
    plt.show()
    
    return roc_auc

import random
def generate_random_walk(graph, root, L):
    """
    :param graph: networkx graph
    :param root: the node where the random walk starts
    :param L: the length of the walk
    :return walk: list of the nodes visited by the random walk
    """
    walk = [root]
    # Fill in the blanks

    node = walk[-1]
    for l in range(L):

      voisins = list(graph.neighbors(node))

      len_list = len(voisins)
      indice = random.randint(0,len_list-1)

      next = voisins[indice]

      walk.append(next)
      node = next

    return walk
def deep_walk(graph, N, L):
    '''
    :param graph: networkx graph
    :param N: the number of walks for each node
    :param L: the walk length
    :return walks: the list of walks
    '''
    walks = []

    ...
    ...

    nodes = list(graph.nodes())

    for i in range(N):

      random.shuffle(nodes)
      
      for node in nodes:

        current_walk = generate_random_walk(graph, node, L)
        walks.append(current_walk)
        
    return walks


#deep_walk(graph, N=3, L=8)

<ipython-input-25-44abedd988bd>:4: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


Upload the graph features for the weighted sampler

In [ ]:
graph_feat = df.copy()
feat_tens = torch.tensor(graph_feat.values)

Laod the data and initialize the dataloaders :

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

import dgl
import dgl.data
from dgl.nn.pytorch.conv import GraphConv, SAGEConv, GATConv
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler,  WeightedRandomSampler
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split

from gensim.test.utils import common_texts
from gensim.models import Word2Vec

dataset = dgl.data.TUDataset('COLLAB')#, directory='~/datasets/TUDataset')
print(f'Dataset: {dataset}:')
print("length of dataset: ", len(dataset))
print('max_num_node : ', dataset.max_num_node)
print('num_labels : ' ,dataset.num_labels)

data = dataset[0] 
print("First graph:")
print(f'Number of nodes: {data[0].num_nodes()}')
print(f'Number of edges: {data[0].num_edges()}')
print(f'Average node degree: {data[0].num_edges() / data[0].num_nodes():.2f}')

# Add a constant node feature.
data[0].ndata['x'] = torch.ones(data[0].num_nodes(), 1)

# Split dataset into training, val and test, shuffling the data first

train_validation_idx, test_idx = train_test_split(list(range(len(dataset))), test_size=.1)
len(train_validation_idx)
train_idx, validation_idx = train_test_split(list(range(len(train_validation_idx))), test_size=.1)

train_validation_dataset=Subset(dataset, train_validation_idx)
test_dataset=Subset(dataset,test_idx)

train_dataset=Subset(train_validation_dataset,train_idx)
validation_dataset=Subset(train_validation_dataset,validation_idx)

num_train = len(train_dataset)
num_test = len(test_dataset)
num_val = len(validation_dataset)


#### Define a weighted sampler to address imbalanced data issue ###

class_sample_count = np.array(
[len(np.where(df.iloc[train_dataset.indices]["labels"] == t)[0]) for t in np.unique(df.iloc[train_dataset.indices]["labels"])])

weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in df.iloc[train_dataset.indices]["labels"]])
samples_weight = torch.from_numpy(samples_weight)  
train_sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight),replacement=True)


class_sample_count = np.array(
[len(np.where(df.iloc[validation_dataset.indices]["labels"] == t)[0]) for t in np.unique(df.iloc[validation_dataset.indices]["labels"])])

weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in df.iloc[validation_dataset.indices]["labels"]])
samples_weight = torch.from_numpy(samples_weight) 
val_sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight),replacement=True)

class_sample_count = np.array([len(np.where(df.iloc[test_dataset.indices]["labels"] == t)[0]) for t in df.iloc[test_dataset.indices]["labels"]])

weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in df.iloc[test_dataset.indices]["labels"]])
samples_weight = torch.from_numpy(samples_weight) 
test_sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight),replacement=True)


##### Node2Vec input ######

num_of_walks= 2
walk_length= 5
w2v_size = 32
embedding_size = w2v_size
window_size = 2
output_filename="graph.embedding"

def Node2Vector(graph,num_of_walks,walk_length,embedding_size,window_size,output_filename):
    # Perform random walks - call function
    walks = deep_walk(dgl.to_networkx(graph), N=num_of_walks, L=walk_length)#
    # Learn representations of nodes - use Word2Vec
    model2V = Word2Vec(sentences=walks, vector_size=embedding_size, window=window_size, min_count=1, workers=4,hs=0) #
    # Save the embedding vectors
    model2V.wv.save_word2vec_format(output_filename)

    node2embedding=model2V.wv
    nodelist = list(dgl.to_networkx(graph))
    word2vec_embeddings = np.asarray([node2embedding[node] for node in nodelist])
    w2v_emb_tensor = torch.from_numpy(word2vec_embeddings)
    return w2v_emb_tensor

def collate(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    graphs, labels = map(list, zip(*samples))
    # Add a constant node feature.
    i=0
    for g in graphs:
        g.ndata['x'] = Node2Vector(g,num_of_walks,walk_length,embedding_size,window_size,output_filename) #torch.ones(g.num_nodes(), 1)
        i+=1
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.tensor(labels)

### GraphSAGE architeture

In [ ]:
# Build a classification model with SAGE convolutions layers
class SAGE(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes, aggregator_type='mean'):
        super().__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, aggregator_type)
        self.conv2 = SAGEConv(h_feats, h_feats, aggregator_type)
        self.dropout = nn.Dropout(0.5)
        self.mlp1 = nn.Linear(h_feats + in_feats, (h_feats + in_feats)//2)
        self.mlp2 = nn.Linear((h_feats + in_feats)//2, (h_feats + in_feats)//4)
        self.mlp3 = nn.Linear((h_feats + in_feats)//4, num_classes)
        self.loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')

        self.bn1 = nn.BatchNorm1d(h_feats)
        self.bn2 = nn.BatchNorm1d(h_feats)
        self.bn3 = nn.BatchNorm1d(h_feats + in_feats)

    def forward(self, g, in_feat):
        m = nn.Softmax(dim=1)
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.bn1(h)
        h = self.conv2(g,h)
        h = F.relu(h)
        h = self.bn2(h)
        h  = torch.cat((feats,h),dim=1)
        h = self.bn3(h)
        with g.local_scope():
            g.ndata['h'] = h
            # Calculate graph representation by average readout.
            hg = dgl.mean_nodes(g, 'h')
        hg = self.mlp1(hg)
        hg = F.relu(hg)
        hg = self.mlp2(hg)
        hg = F.relu(hg)
        hg = self.mlp3(hg)
        hg = m(hg)
        return hg

# Hyperparameters
in_feats = embedding_size 
h_feats = 128
batch_size = 64

device = ("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

# DataLoader
train_loader = GraphDataLoader(train_dataset, batch_size=batch_size, collate_fn=collate, sampler=train_sampler)
validation_loader = GraphDataLoader(validation_dataset, batch_size=batch_size,collate_fn=collate, sampler=val_sampler)
test_loader = GraphDataLoader(test_dataset, batch_size=batch_size, collate_fn=collate, sampler=test_sampler)


import transformers

# Create the model
lr = 1e-3

model = SAGE(in_feats, h_feats, dataset.num_labels).to(device)
opt = torch.optim.Adam(model.parameters(), lr=lr)

num_epochs = 100
num_training_steps = num_epochs * len(train_loader)

#define a learning scheduler
lr_scheduler = transformers.get_scheduler(
    name="linear", optimizer=opt, num_warmup_steps=0, num_training_steps=num_training_steps)

# Training loop
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []

for epoch in range(num_epochs):
    model.train()
    train_losses = []
    train_correct = 0
    for batched_graph, labels in tqdm(train_loader):
        batched_graph = batched_graph.to(device)
        feats = batched_graph.ndata['x'].float().to(device)

        labels = labels.long().to(device)
        logits = model(batched_graph, feats)

        loss = model.loss_fn(logits, labels)
        opt.zero_grad()
        loss.backward()
        opt.step()
        lr_scheduler.step()

        train_losses.append(loss.detach().item())
        pred = logits.argmax(1)
        train_correct += (pred == labels).sum().item()
    epoch_train_acc = train_correct/num_train

    model.eval()
    val_losses = []
    val_correct = 0
    for batched_graph, labels in validation_loader:
        batched_graph = batched_graph.to(device)
        feats = batched_graph.ndata['x'].float().to(device)
        labels = labels.long().to(device)

        with torch.no_grad():
            logits = model(batched_graph, feats)
            loss = model.loss_fn(logits, labels)
            val_losses.append(loss.detach().item())
        pred = logits.argmax(1)
        val_correct += (pred == labels).sum().item()
    epoch_val_acc = val_correct/num_val

    train_loss_history.append(np.mean(train_losses))
    train_acc_history.append(epoch_train_acc)
    val_loss_history.append(np.mean(val_losses))
    val_acc_history.append(epoch_val_acc)

    print(f'Epoch {epoch} | Train Loss {np.mean(train_losses):.4f} | Train accuracy {epoch_train_acc}')
    print(f'Epoch {epoch} | Val Loss {np.mean(val_losses):.4f} | Val accuracy {epoch_val_acc}')

# Test loop
model.eval()
test_correct = 0
for batched_graph, labels in tqdm(test_loader):
    batched_graph = batched_graph.to(device)
    feats = batched_graph.ndata['x'].float().to(device)
    labels = labels.long().to(device)

    with torch.no_grad():
        logits = model(batched_graph, feats)
        pred = logits.argmax(1)
        test_correct += (pred == labels).sum().item()

test_acc = test_correct / num_test
print(f'Test accuracy {test_acc:.4f}')

# Plot the losses
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.plot(train_loss_history, label="Train loss")
plt.plot(val_loss_history, label="Val loss")
plt.legend()
plt.subplot(1,2,2)
plt.plot(train_acc_history, label="Train acc")
plt.plot(val_acc_history, label="Val acc")
plt.legend()

## GATConv

In [ ]:
# Build a classification model with GAT convolutions layers
class GAT(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes, heads = 4):
        super().__init__()
        self.conv1 = GATConv(in_feats, h_feats, num_heads = heads, residual = True)
        self.conv2 = GATConv(h_feats, h_feats,num_heads = heads, residual = True)
        self.dropout = nn.Dropout(0.5)
        self.mlp1 = nn.Linear(h_feats, (h_feats)//2)
        self.mlp2 = nn.Linear((h_feats)//2, (h_feats)//4)
        self.mlp3 = nn.Linear((h_feats)//4, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv2(g, h)
        h = F.relu(h)
        with g.local_scope():
            g.ndata['h'] = h
            # Calculate graph representation by average readout.
            hg = dgl.mean_nodes(g, 'h')
        hg = self.mlp1(hg)
        hg = F.relu(hg)
        hg = self.mlp2(hg)
        hg = F.relu(hg)
        hg = self.mlp3(hg)
        return hg

# Hyperparameters
in_feats = embedding_size 
h_feats = 128

batch_size = 64

device = ("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

# DataLoader

train_loader = GraphDataLoader(train_dataset, batch_size=batch_size, collate_fn=collate, sampler=train_sampler)
print(train_loader)
validation_loader = GraphDataLoader(validation_dataset, batch_size=batch_size,collate_fn=collate, sampler=val_sampler)
test_loader = GraphDataLoader(test_dataset, batch_size=batch_size, collate_fn=collate, sampler=test_sampler)


import transformers

# Create the model
lr = 1e-3

model = GAT(in_feats, h_feats, dataset.num_labels).to(device)

opt = torch.optim.Adam(model.parameters(), lr=lr)

num_epochs = 10

#optimizer = Adam(model.parameters(), lr=1e-6)


num_training_steps = num_epochs * len(train_loader)

lr_scheduler = transformers.get_scheduler(
    name="linear", optimizer=opt, num_warmup_steps=0, num_training_steps=num_training_steps)

# Training loop
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []

for epoch in range(num_epochs):
    model.train()
    train_losses = []
    train_correct = 0
    for batched_graph, labels in tqdm(train_loader):
        batched_graph = batched_graph.to(device)
        feats = batched_graph.ndata['x'].float().to(device)
        labels = labels.long().to(device)
        logits = model(batched_graph, feats)
        logits = torch.mean(logits,dim=1)
        logits = torch.mean(logits,dim=1)
        loss = F.cross_entropy(logits, labels)
        opt.zero_grad()
        loss.backward()
        opt.step()
        lr_scheduler.step()
        train_losses.append(loss.detach().item())
        pred = logits.argmax(1)
        train_correct += (pred == labels).sum().item()
    epoch_train_acc = train_correct/num_train

    model.eval()
    val_losses = []
    val_correct = 0
    for batched_graph, labels in validation_loader:
        batched_graph = batched_graph.to(device)
        feats = batched_graph.ndata['x'].float().to(device)
        labels = labels.long().to(device)
        with torch.no_grad():
            logits = model(batched_graph, feats)
            logits = torch.mean(logits,dim=1)
            logits = torch.mean(logits,dim=1)
            loss = F.cross_entropy(logits, labels)
            val_losses.append(loss.detach().item())
        pred = logits.argmax(1)
        val_correct += (pred == labels).sum().item()
    epoch_val_acc = val_correct/num_val

    train_loss_history.append(np.mean(train_losses))
    train_acc_history.append(epoch_train_acc)
    val_loss_history.append(np.mean(val_losses))
    val_acc_history.append(epoch_val_acc)

    print(f'Epoch {epoch} | Train Loss {np.mean(train_losses):.4f} | Train accuracy {epoch_train_acc}')
    print(f'Epoch {epoch} | Val Loss {np.mean(val_losses):.4f} | Val accuracy {epoch_val_acc}')

# Test loop
model.eval()
test_correct = 0
for batched_graph, labels in tqdm(test_loader):
    batched_graph = batched_graph.to(device)
    feats = batched_graph.ndata['x'].float().to(device)
    labels = labels.long().to(device)
    with torch.no_grad():
        logits = model(batched_graph, feats)
        logits = torch.mean(logits,dim=1)
        logits = torch.mean(logits,dim=1)
        pred = logits.argmax(1)
        test_correct += (pred == labels).sum().item()

test_acc = test_correct / num_test
print(f'Test accuracy {test_acc:.4f}')

# Plot the losses
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.plot(train_loss_history, label="Train loss")
plt.plot(val_loss_history, label="Val loss")
plt.legend()
plt.subplot(1,2,2)
plt.plot(train_acc_history, label="Train acc")
plt.plot(val_acc_history, label="Val acc")
plt.legend()

## Hybrid

Let's define our class

In [ ]:
# Import confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Take the previous trained model, and fit a ML on top of it instead of the mlp
class Hybrid(nn.Module):
    def __init__(self, gnn, clf):
        super(Hybrid, self).__init__()
        self.gnn = gnn
        self.clf = clf
        self.train_features = None
        self.train_labels = None
        self.val_features = None
        self.val_labels = None
        self.test_features = None
        self.test_labels = None

    def cropped_foward_gnn(self, g, in_feat):
        """Same as the GNN forward function but without the mlp part"""
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv2(g, h)
        h = F.relu(h)

        with g.local_scope():
            g.ndata['h'] = h
            # Calculate graph representation by average readout.
            hg = dgl.mean_nodes(g, 'h')
    
    def build_features_array(self, dataloader):
        features = []
        for batched_graph, labels in dataloader:
            graphs = dgl.unbatch(batched_graph)
            # Then, for each graph, we extract the node features and the label
            for graph in graphs:
                gnn_output = self.cropped_foward_gnn(batched_graph, batched_graph.ndata['x'].float())
                gnn_output = torch.mean(gnn_output, dim=1)
                gnn_output = torch.mean(gnn_output, dim=1)
                features.append(gnn_output.detach().numpy())
        return np.concatenate(features)

    def build_features_and_labels_array(self, dataloader):
        features = []
        labels = []
        for batched_graph, batched_label in tqdm(dataloader):
            # First, unbatch the batched graph
            graphs = dgl.unbatch(batched_graph)
            # Then, for each graph, we extract the node features and the label
            for graph, graph_label in zip(graphs, batched_label):
                # Extract the node features
                gnn_output = self.cropped_foward_gnn(graph, graph.ndata['x'].float())
                gnn_output = torch.mean(gnn_output, dim=1)
                gnn_output = torch.mean(gnn_output, dim=1)
                features.append(gnn_output.detach().numpy())
                # Extract the label
                labels.append(graph_label.detach().numpy())
        
        return np.concatenate(features), np.array(labels)

    def train_clf(self, dataloader):
        features, labels = self.build_features_and_labels_array(dataloader)
        self.clf.fit(features, labels)
    
    def predict_clf(self, dataloader):
        features = self.build_features_array(dataloader)
        return self.clf.predict(features)

    def evaluate_clf(self, dataloader):
        features, labels = self.build_features_and_labels_array(dataloader)
        # Plot confusion matrix
        cm = confusion_matrix(labels, self.clf.predict(features))
        plt.figure(figsize=(10,10))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Confusion Matrix")
        return self.clf.score(features, labels)
        


Test the approach on a Random Forest

In [ ]:
# Create the model
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

hybrid_model = Hybrid(model.to("cpu"), clf)#.to(device)
hybrid_model.train_clf(train_loader)
hybrid_model.evaluate_clf(test_loader)

Benchmark three models

In [ ]:
clf_benchmark = [RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0),
                 SVC(gamma='auto'),
                 XGBClassifier()]

for clf in clf_benchmark:
    hybrid_model = Hybrid(model.to("cpu"), clf)
    hybrid_model.train_clf(train_loader)
    print(hybrid_model.evaluate_clf(test_loader))
